In [69]:
using ForwardDiff

In [70]:
using ForwardDiff

function newton_method(f, x0; tol=1e-8, max_iter=100)
    """
    Find the root of a 1D function using Newton's method with automatic differentiation.

    Parameters:
    f (Function): The function for which to find the root.
    x0 (Real): Initial guess for the root.
    tol (Real): Tolerance for convergence (default: 1e-8).
    max_iter (Int): Maximum number of iterations (default: 100).

    Returns:
    Real: The estimated root.
    Int: Number of iterations performed.
    Bool: Whether the method converged.
    Vector{Float64}: Array of successive guesses for the root.
    """
    x = x0
    guesses = Float64[]  # Initialize an empty array to store guesses
    # push!(guesses, x0)   # Include the initial guess in the array

    for i in 1:max_iter
        # Compute the function value and its derivative using ForwardDiff
        f_x = f(x)
        f_prime_x = ForwardDiff.derivative(f, x)

        # Check if the derivative is zero to avoid division by zero
        if abs(f_prime_x) < tol
            println("Derivative is too close to zero. Stopping iteration.")
            return x, i, false, guesses
        end

        # Newton's update
        x_new = x - f_x / f_prime_x

        # Store the new guess in the array
        push!(guesses, x_new)

        # Check for convergence
        if abs(x_new - x) < tol
            return x_new, i, true, guesses
        end

        # Update x
        x = x_new
    end

    # If max_iter is reached without convergence
    println("Maximum iterations reached. Stopping.")
    return x, max_iter, false, guesses
end

newton_method (generic function with 1 method)

In [71]:
f(x) = 0.2 + x^2 - 3*x*cos(2*x);
g(x) = 0.1 + x^3 - 2*x*sin(5*x);

In [ ]:
fx0 = 1;
# gx0 = -0.625;

# Call Newton's method
root, iterations, converged, guesses = newton_method(f, fx0)

# Print results
println("Estimated root: ", root)
println("Number of iterations: ", iterations)
println("Converged: ", converged)
# println("Successive guesses for root: ", guesses)

Estimated root: 0.6251564275801937
Number of iterations: 6
Converged: true
Successive guesses for root: [0.7187066637427939, 0.6378011291931867, 0.6254657277759242, 0.6251566222683084, 0.625156427580271, 0.6251564275801937]


In [73]:
function shanks_transformation_table(S)
    """
    Compute the Shanks transformation for a sequence of partial sums S and print the results as a table.

    Parameters:
    S (Vector{<:Real}): A vector of partial sums of a series.

    Returns:
    Nothing (prints the table directly).
    """
    n = length(S)
    transformed_sequences = Vector{Vector{Real}}()
    push!(transformed_sequences, S)  # Level 0: original sequence

    # Apply Shanks transformation iteratively
    while true
        current_sequence = transformed_sequences[end]
        m = length(current_sequence)
        if m < 3
            break  # Need at least 3 terms to compute the transformation
        end

        # Compute the next level of Shanks transformation
        next_sequence = Real[]
        for i in 1:(m - 2)
            numerator = current_sequence[i+2] * current_sequence[i] - current_sequence[i+1]^2
            denominator = current_sequence[i+2] + current_sequence[i] - 2 * current_sequence[i+1]
            if denominator == 0
                push!(next_sequence, NaN)  # Avoid division by zero
            else
                push!(next_sequence, numerator / denominator)
            end
        end

        push!(transformed_sequences, next_sequence)
    end

    # Print the table
    max_length = maximum(length(seq) for seq in transformed_sequences)
    println("Shanks Transformation Table:")
    println("--------------------------------------------------------------")
    for i in 1:max_length
        for seq in transformed_sequences
            if i <= length(seq)
                # Convert the number to a string and pad it for alignment
                value_str = string(round(seq[i], digits=8))
                print(lpad(value_str, 12), "  ")
            else
                print(" " ^ 12, "  ")  # Padding for alignment
            end
        end
        println()
    end
    println("--------------------------------------------------------------")
end

shanks_transformation_table (generic function with 1 method)

In [74]:
#=
# Compute partial sums of the series
function compute_partial_sums(n)
    S = Float64[]
    partial_sum = 0.0
    for k in 0:(n-1)
        partial_sum += (-1)^k / (2k + 1)
        push!(S, 4 * partial_sum)
    end
    return S
end

# Generate partial sums
partial_sums = compute_partial_sums(10)

# Print the Shanks transformation table
shanks_transformation_table(partial_sums)
=#

In [87]:
shanks_transformation_table(guesses[2:5])

Shanks Transformation Table:
--------------------------------------------------------------
  0.63780113    0.62514868  
  0.62546573    0.62515643  
  0.62515662                
  0.62515643                
--------------------------------------------------------------


In [76]:
function bisection(f, a, b; tol=1e-6, max_iter=100)
    # Check if the initial interval has a root
    if f(a) * f(b) >= 0
        error("Function must have opposite signs at interval endpoints")
    end
    
    # Initialize iteration counter and interval endpoints
    left = a
    right = b
    f_left = f(left)
    f_right = f(right)
    
    #println("\nStarting bisection method with interval [", left, ", ", right, "]")
    #println("f(left) = ", f_left)
    #println("f(right) = ", f_right)
    
    # Track the width of our interval for convergence checking
    interval_width = right - left

    guesses = Float64[]
    

    for iter in 1:max_iter
        # Calculate interval information
        interval_width = right - left
        mid = (left + right) / 2
        push!(guesses, mid)

        f_mid = f(mid)
        
        # Print current iteration details
        #println("\nIteration ", iter, ":")
        #println("Current interval: [", left, ", ", right, "]")
        #println("Interval width: ", interval_width)
        #println("Midpoint = ", mid)
        #println("f(midpoint) = ", f_mid)
        
        # Check if we found exact root (unlikely with floating-point)
        if f_mid == 0
            #println("\nExact root found!")
            return mid, guesses
        end
        
        # Determine which half of interval contains the root
        if f_left * f_mid < 0
            # Root is in left half
            right = mid
            #println("Root is in left half - updating right endpoint")
        else
            # Root is in right half
            left = mid
            f_left = f_mid
            #println("Root is in right half - updating left endpoint")
        end
        
        # Check for convergence based on interval width
        if interval_width < tol
            #println("\nConverged! Final interval width ", interval_width, " is less than tolerance ", tol)
            final_approximation = (left + right) / 2
            #println("Final approximation: ", final_approximation)
            #println("f(approximation) = ", f(final_approximation))
            return final_approximation, guesses
        end
    end
    
    # If we reach here, we've hit max iterations without converging
    @warn "Maximum iterations reached. The method may not have converged."
    return (left + right) / 2, guesses
end

bisection (generic function with 3 methods)

In [77]:
root, guesses_bi = bisection(f, 0.5, 1.5)
# root, guesses_bi = bisection(g, -1, -0.25)

(0.6251566410064697, [1.0, 0.75, 0.625, 0.6875, 0.65625, 0.640625, 0.6328125, 0.62890625, 0.626953125, 0.6259765625  …  0.625244140625, 0.6251220703125, 0.62518310546875, 0.625152587890625, 0.6251678466796875, 0.6251602172851562, 0.6251564025878906, 0.6251583099365234, 0.625157356262207, 0.6251568794250488])

In [85]:
shanks_transformation_table(guesses_bi[2:6])

Shanks Transformation Table:
--------------------------------------------------------------
        0.75    0.66666667    0.66666667  
       0.625    0.66666667                
      0.6875         0.625                
     0.65625                              
    0.640625                              
--------------------------------------------------------------
